## IMPORTING LIBRARIES


In [12]:
import urllib2
from bs4 import BeautifulSoup
import requests
import numpy as np 
import time
import re

## Creating Fake Header in  order to parse the website

In [22]:
headers = {
'User-Agent': 'Mozilla/4.0 (Macintosh; Intel Mac OS X 10.13; rv:60.0) Gecko/20100101 Firefox/60.0'}


## Few functions for PageVistits , FindingUrl, Finding specific class,finding Url

In [23]:
#Function for visiting a URL and returning the full page
def pagevists(url):
    html_content = requests.get(url, headers=headers)
    html_content.status_code
    data=html_content.text
    soup=BeautifulSoup(data,"lxml")
    return soup

In [24]:
#A program in which we can pass arguments as the full page and what to find and the tags
def find(data,find,form,**kwargs):
    return data.findAll(find,form)
#This function finds all the links present in a page or in a piece of html tags , basically I have 
#build this to find inside a block of html tags passed as list 
def linkfinder(val):
    url=[]
    for link in range (len(val)):
        l=val[link]
        k=l.find('a')
        url.append(k.get('href'))
    return(url)
#This function builds the broken url so that we can visit the page 
def urlbuilder(val):
    
    for i in range(len(val)):
        val[i]='https://health.usnews.com'+val[i]
    return val

In [25]:
#Requesting the page 
url = 'https://health.usnews.com/doctors/city-index/new-jersey'
soup=pagevists(url)
#Here I am using Multiple Fuction in one go i.e 
#first getting the links block
#then find the the links 
#finally rebuilding the links 
#here set and list is used to avoid any repetition

#All the links for city 
citylinks=list (set (urlbuilder(linkfinder(find(soup,'li',{'class':'List__ListItem-dl3d8e-1 kfaWAY'})))))


## As we have found all the links now we will visit each link and also simultanouesly start building the elasticsearch documents of each doctor first we will create some more functions

The following piece of code was to add  Load More functionality somehow I was getting blocked everytime and as beautiful soup doesnot support page clicks and hence it was not possibe,
I tried to open via proxy server , it worked fine but still integrating it to python was difficult ,Using 
of selenium in this case should have solved the problem

In [26]:
# def doc_finder(data,link):
#     l=[]
#     import sys
#     m=1
    
#     while(m):
            
#             m=data.find('a',{'rel':'next' })
        
#                 url='http://health.usnews.com'+m.get('href')
#             except:
#                 url=link
            
#             l.append(url)
#             data=pagevists(url)
#     if len(l)==0:
#         data=pagevists(l[0])
#         return data
#     else:
#         data=pagevists(l[len(l)-2])
#         return data
    

In [27]:
#This function is used in the doctor's list page where this will find the links of all doctor in that page 
def doc_page(val):
    doc=[]
    for link in range (len(val)):
        doc.append(val[link].get('href'))
    return doc

In [28]:
#These are all the function which are scrapping data from the doctor's page and I have used regex to remove blank
#spaces and also to remove \n
#All these function is finding the class of the tag and returning the text format

def name_scrap(data):
    name=data.find_all('h1',{'class':'hero-heading flex-media-heading block-tight doctor-name '})
    for i in name :
        return re.sub("\s\s+" , " ", i.text)
def speciality(data):
    name=data.find_all('a',{'class':'bar-flush '})
    for i in name :
        return re.sub("\s\s+" , " ", i.text)
def city(data):
    name=data.find_all('strong')
    for i in name :
        return re.sub("\s\s+" , " ", i.text)
def overview(data):
    name=data.find_all('div',{'class':'block-normal clearfix'})
    for i in name :
        return re.sub("\s\s+" , " ", i.text)
def experiance(data):
    name=data.find_all('span',{'class':'text-large heading-normal-for-small-only right-for-medium-up'})
    lm=[]
    for i in name:
        lm.append(i.text)
    return (lm[1])
def languages(data):
    name=data.find_all('span',{'class':'text-large heading-normal-for-small-only right-for-medium-up text-right showmore'})
    for i in name :
        return re.sub("\s\s+" , " ", i.text)
def Address(data):
    name=data.find_all('span',{'data-js-id':'doctor-address'})
    for i in name :
        return re.sub("\s\s+" , " ", i.text)
def affiliation(data):
    name=data.find_all('a',{'data-tracking-placement':'hospital-affiliation-in-content'})
    for i in name :
        return re.sub("\s\s+" , " ", i.text)
def PinFinder(data):
    l=Address(data)
    lm=l.split()
    return lm[-1]


In [29]:
#Importing Elastic Search and connecting it with the local host 
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost','port': 9200}])

##  This is the code block which is doing the whole scrapping and storing each document as a elasticsearch document : below is the detailed explanation

step1:  The first for loop is visiting the city in a iterative way, and as the url contains which city we are 
visiting hence I am spliting it and storing as list


step2: Now I have visited the City page , and also scrapped all the specialised links from there 

step3: second loop is traversing each specialisation link one by one and getting the links of all doctors from there

step4:As I have the doctors links I visited the links iterativly and scrapped the necessary data ,

step5:Finally I  created a json document and pushed that into elasticsearch 

In [38]:
n=1
from IPython.display import clear_output
#The range should be the length of citylinks ,here only 5 is being taken for time limitation
for i in range(5):
    soup=pagevists(citylinks[i])
    soup=pagevists(citylinks[i])
    c=citylinks[i].split('/')
#Used for reducing load time on the server 
    time.sleep(np.random.choice([1,4,2]))

#Finding all the specialisation links of a city    
    next_page=list (set (urlbuilder(linkfinder(find(soup,'li',{'class':'List__ListItem-dl3d8e-1 kfaWAY'})))))
    sp=next_page[i].split('/')
#The range should be the length of next_page ,only 1 is taken again for time limitation
    for j in range(1):
        data=pagevists(next_page[j])
#Used for reducing load time on the server 
        time.sleep(np.random.choice([1,4,2]))
        doc_index=[]
#Getting all the links of doctor's
        doc_index.append(urlbuilder(doc_page(find(data,'a',{'class':'search-result-link bar-tighter'}))))
#For converting list of list to only list
        doc_index=doc_index[0]
        for k in range (len(doc_index)):
            
            data=pagevists(doc_index[k])
            time.sleep(np.random.choice([1,4,2]))
#Creating Json Document 
            document={
                'Doctor_Name':name_scrap(data),
                'Speciality':speciality(data),
                'City':c[-1],
                'overview':overview(data),
                'Experiance':experiance(data),
                'Languages':languages(data),
                'Location':Address(data),
                'Affiliation':affiliation(data),
                'Pin':PinFinder(data)
            }
#Finally Pushing it 
            es.index(index='doc_profile',doc_type='docs',id=n,body=document)
#Just for checking the counts and if the above code works properly
            clear_output()
            n=n+1
            print (n)
            
        
    

83


### Verification if entered properly or not 

In [39]:
es.get(index='doc_profile',doc_type='docs',id=1)

{u'_id': u'1',
 u'_index': u'doc_profile',
 u'_source': {u'Affiliation': None,
  u'City': u'jamesburg',
  u'Doctor_Name': u' Dr. Stuart Rubenstein MD\n',
  u'Experiance': u'21+',
  u'Languages': u'\nEnglish\n',
  u'Location': u' 29 Harwood Road Monroe Township, NJ 08831 ',
  u'Pin': u'08831',
  u'Speciality': u'Internal Medicine',
  u'overview': u' Dr. Stuart Rubenstein is an internist in Monroe Township, New Jersey. He received his medical degree from State University of New York Downstate Medical Center College of Medicine and has been in practice for more than 20 years. '},
 u'_type': u'docs',
 u'_version': 1,
 u'found': True}

# Information Retrieval using Elasticsearch


In [40]:
from elasticsearch_dsl.connections import connections
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

# Define a default Elasticsearch client
client = connections.create_connection(hosts=['http://localhost:9200'])
#Defining the elasticsearch Search method
s = Search(using=client, index='doc_profile')


### Summary Of the Data  

###  1. Finding Doctors per city 

In [41]:
s.aggs.bucket('Doc_City', 'terms', field='City.keyword', size=1000)
t=s.execute()
for i in t.aggregations.Doc_City.buckets:
    print i.key , i.doc_count

elizabeth 21
fair-lawn 20
jamesburg 20
hightstown 12
kinnelon 9


### 2. Finding Number of Doctors by speciality

In [42]:
s = Search(using=client, index='doc_profile')
s.aggs.bucket('Speciality', 'terms', field='Specialitry.keyword', size=1000)
t=s.execute()
for i in t.aggregations.Speciality.buckets:
    print i.key , i.doc_count

### 3. Finding Number of doctors by Experiance  

In [43]:
s = Search(using=client, index='doc_profile')
s.aggs.bucket('Exp', 'terms', field='Experiance.keyword', size=1000)
t=s.execute()
for i in t.aggregations.Exp.buckets:
    print i.key , i.doc_count

21+ 54
11 - 20 17
6 - 10 7
1 - 2 2
3 - 5 1
Male 1


### 4. Finding the number of doctors by PIN

In [44]:
s = Search(using=client, index='doc_profile')
s.aggs.bucket('PIN', 'terms', field='Pin.keyword', size=1000)
t=s.execute()
for i in t.aggregations.PIN.buckets:
    print i.key , i.doc_count

08831 18
10032 10
07601 7
07202 5
07208 4
07103 3
07111 3
07112 3
08628 3
07503 2
08625 2
07016 1
07039 1
07054 1
07083 1
07204 1
07207 1
07666 1
07677 1
07722 1
07746 1
07856 1
08540 1
08542 1
08558 1
08618 1
08857 1
08884 1
10022 1
10039 1
10467 1
19067 1
19127 1


### Note : I can only retrieve data from the 1st page of doctors's List ie. https://health.usnews.com/doctors/anesthesiologists/new-jersey/absecon , As the Load More link was not opening by any python native packages I researched and found It can be  opened by a proxy server , but I do not have access to a proxy server , third party libraries could have solve the problem



<h1 align='Center'> The end <h1>